In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from simulation.student import LSTMPredictor
from simulation.simulator import FDTDSimulator

from dataset import load_core_set_data

# 加载模型

In [ ]:
save_dir = "data/model/lstm_model.pth"
model_data = torch.load(save_dir)

model = LSTMPredictor(input_size=10, hidden_size=64, num_layers=1, output_size=ports)
model.load_state_dict(model_data["model_state_dict"])
model.eval()

radius_matrix = model_data["radius_matrix"]

simulator = FDTDSimulator(radius_matrix=radius_matrix)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.MSELoss()


# 加载数据集

In [ ]:
class FDTDDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.FloatTensor(data)
        self.labels = torch.FloatTensor(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


train_data, train_labels, test_data, test_labels = load_core_set_data()

train_dataset = FDTDDataset(train_data, train_labels)
test_dataset = FDTDDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        _, pred_seq = model.get_sequence_output(inputs)
        target_seq = simulator(inputs)
        loss = criterion(pred_seq, target_seq)
        print(f"Loss: {loss.item()}")

